In [3]:
!pip install langchain langchain-community pandas ollama tqdm


[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [4]:
from langchain_community.llms import Ollama
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence

import logging
import time
import pandas as pd
import logging
import time

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [5]:
PROMPT_TEMPLATE = """Instruction: Analyze the following review text and provide one distinct outputs formatted in JSON:

1. **Sentiment Classification:** Indicate whether the sentiment of the review is Positive, Neutral, or Negative.
2. **Named Entity Extraction:** List all named entities present in the text, categorizing them by type (e.g., people, locations, organizations, etc.).
3. **Required JSON Format:** Ensure the response is formatted in JSON according to the following schema:

{{
  "sentiment": "<sentiment>",
  "entities": {{
    "person": ["<person_name1>", "<person_name2>", ...],
    "location": ["<location1>", "<location2>", ...],
    "organization": ["<organization1>", "<organization2>", ...],
    "product": ["<product1>", "<product2>", ...]
  }}
}}

example:

"I recently visited the restaurant 'La Dolce Vita' in Rome and was thrilled with the service and food. The waiter, Marco, was exceptionally friendly and the truffle risotto was simply divine. I can't wait to return and recommend this place to my friends."

```json

{{
  "sentiment": "Positive",
  "entities": {{
    "person": ["Marco"],
    "location": ["Rome"],
    "organization": ["La Dolce Vita"],
    "product": ["truffle risotto"]
  }}
}}

```
{content}





"""

In [6]:
def process_review(review: str, chain: RunnableSequence) -> str:
    """
    It processes the review text and returns the LLM response as string.
    
    Arguments:
        review (str): The review text.
        chain (RunnableSequence): The LLM chain.
        
    Return:
        The LLM response as string.
    """
    try:
        return chain.invoke({"content": review})
    except Exception as e:
        logging.error(f"Error invoking the chain: {e}")
        return None

In [11]:
from tqdm import tqdm

def call_model_llm(model_name: str, output_file_path: str) -> pd.DataFrame:
    """
    It calls the LLM model using Ollama. It returns the sampled dataframe enriched with relevant columns.
    
    Arguments:
        model_name: The name of the model to invoke via Ollama.
    
    Return:
        The enriched dataframe.
    """
    dataframe: pd.DataFrame = pd.read_csv(output_file_path)
    chain: RunnableSequence = RunnableSequence(
        PromptTemplate.from_template(PROMPT_TEMPLATE) | Ollama(model=model_name)
    )
    dataframe: pd.DataFrame = dataframe[dataframe.output == "$$$"]
    total_rows: int = len(dataframe.index)
    for i in tqdm(range(total_rows), total=total_rows):
        row = dataframe.iloc[i]
        # logging.info(f"Processing row {i + 1} out of {total_rows}")
        dataframe.loc[i, "output"] = process_review(row["review"], chain)
        dataframe.to_csv(output_file_path)
    return dataframe

In [ ]:
import os

# Define the model name to use
MODEL_NAME: str = "phi3:medium"

# Define the output path and get the result as csv
output_file_path: str = f"../resources/sampled_reviews_with_output_{MODEL_NAME}.csv"

exists: bool = os.path.exists(output_file_path)
if not exists:
    sampled: pd.DataFrame = pd.read_csv("../resources/IMDB Dataset Sampled.csv")
    sampled["output"] = sampled.apply(lambda row: "$$$", axis=1)
    sampled.to_csv(output_file_path, index=False)

# Produce a dataframe by invoking the chain
output: pd.DataFrame = call_model_llm(model_name=MODEL_NAME, output_file_path=output_file_path)

output.to_csv(output_file_path, index=False)

logging.info("Completed processing.")

  0%|          | 4/1056 [00:14<1:03:19,  3.61s/it]